In [1]:
import re
import urllib
import urllib.request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
%matplotlib inline
plt.style.use('ggplot')
sns.set_style('darkgrid')

In [2]:
# embaralhei a lista de players pra nao pegar só de GM - NAO PRECISA RODAR ESSA CEDULA DNV
# shuffled_players.to_csv('shuffled_player_list_chesscom.csv',index = False)

In [3]:
players = pd.read_csv('shuffled_player_list_chesscom.csv')

In [4]:
players

,username,title,bullet_rating,blitz_rating,rapid_rating
0,michaelq2d5,GM,2793,2765,2903
1,bernd_stromberg,CM,2619,2701,0
2,g2_coder,FM,2350,2200,0
3,mennabarreto,FM,2251,2407,0
4,radojeiriz,FM,2388,2271,2489
...,...,...,...,...,...
8556,skiiier,FM,2492,2477,0
8557,volodymyrmolyboha,NM,2431,2503,2189
8558,miodrag-jevtic,FM,2378,2356,1936
8559,jerryle,NM,2214,2251,1365


In [5]:
# resolve erro "This event loop is already running"
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [6]:
# instala e importa https://github.com/sarartur/chess.com
# wrapper para dados publicados pelo chess.com
!pip install chessdotcom
import chessdotcom

ERROR: Could not find a version that satisfies the requirement chessdotcom
ERROR: No matching distribution found for chessdotcom


In [26]:
# peguei um role do stack overflow e editei, pega os anos/meses que teve jogo de cada pessoa

def getDates(user):
    username = user #change 
    baseUrl = "https://api.chess.com/pub/player/" + username + "/games/"
    archivesUrl = baseUrl + "archives"

    #read the archives url and store in a list
    f = urllib.request.urlopen(archivesUrl)
    archives = f.read().decode("utf-8")
    archives = archives.replace("{\"archives\":[\"", "\",\"")
    archivesList = archives.split("\",\"" + baseUrl)
    archivesList[len(archivesList)-1] = archivesList[len(archivesList)-1].rstrip("\"]}")
    #print(archivesList)
    month = []
    year = []
    # trata caso em que o jogador nao tem jogos
    if archivesList[0] == '{"archives":[':
        print('Erro - Jogador não possui partidas registradas')
        return(month, year) 
    archivesList.remove('')
    for i in archivesList:
        year.append(int(i[0:4]))
        month.append(int(i[5:]))
    return(month, year)

In [8]:
games_db = pd.read_csv('empty_games.csv')
games_db.head()

,id,ranked,time_control,status,winner,white,black,moves


In [9]:
def checkstatus(white_result, black_result):
    if white_result != 'win':
        return white_result
    else:
        return black_result
    
def checkwinner(white, black):
    if white['result'] == 'win':
        return white['username']
    elif black['result'] == 'win':
        return black['username']
    else:
        return 'draw'

def trataPGN(pgn):
    # remove clock
    txt = '{\[%clk \d+:\d+:\d+(\.\d+)?\]}'
    a = re.sub(txt, "", pgn)

    # remove header
    txt = '\[.+\]'
    a = re.sub(txt, "", a)
    a

    # remove \n
    txt = '\\n'
    a = re.sub(txt, "", a)
    a
    
    return(a)    

In [51]:
# escolhe que jogadores voce quer pegar
players_to_db = players[45:55]
players_to_db

,username,title,bullet_rating,blitz_rating,rapid_rating
45,1d51-0,NM,2321,2072,0
46,sl0m,NM,1594,2117,2140
47,jonahwillow,FM,3171,2832,2179
48,kosanovicgoran,GM,0,2636,2028
49,johangunawan,FM,1951,2056,0
50,gmpeter008,GM,2390,2353,2057
51,oldturkishboy,NM,0,0,0
52,lordiron,FM,2561,2293,2431
53,lpsupi,GM,2775,2888,2707
54,maximych26,FM,2150,2277,2036


In [52]:
games_db = pd.read_csv('empty_games.csv')

for k in players_to_db['username']:
    month, year = getDates(k)
    print(k)
    for i in range(len(month)):
        response = chessdotcom.get_player_games_by_month(k,month=month[i],year=year[i]).json
        for j in (response['games']):
            try:
                temp = {
                    'id':j['uuid'],
                    'ranked':j['rated'],
                    'time_control': j['time_class'],
                    'status': checkstatus(j['white']['result'],j['black']['result']),
                    'winner': checkwinner(j['white'],j['black']),
                    'white': (j['white']['username']),
                    'black': (j['black']['username']),
                    'moves': trataPGN(j['pgn'])
                }
                #print((j['white']['result'],j['black']['result']))
                games_db = games_db.append(temp,ignore_index=True)
            except:
                temp = 0

1d51-0
sl0m
jonahwillow
kosanovicgoran
johangunawan
gmpeter008
Erro - Jogador não possui partidas registradas
oldturkishboy
lordiron
lpsupi
maximych26


In [53]:
games_db

,id,ranked,time_control,status,winner,white,black,moves
0,a42ab65e-1fca-11ec-9f2a-09ff3c010001,True,blitz,checkmated,Crackitovich,1d51-0,Crackitovich,1. d4 1... d5 2. c4 2... e6 3. Nc3 3... N...
1,5580a8b7-1fcb-11ec-9f2a-09ff3c010001,True,blitz,resigned,1d51-0,Crackitovich,1d51-0,1. e4 1... c5 2. Nf3 2... g6 3. d4 3... c...
2,ada9c8b9-1fcb-11ec-9f2a-09ff3c010001,True,blitz,resigned,1d51-0,1d51-0,walidsaid73,1. Nf3 1... d5 2. g3 2... e6 3. Bg2 3... ...
3,5ef77e0c-1fcc-11ec-9f2a-09ff3c010001,True,blitz,checkmated,ivanmanshm,ivanmanshm,1d51-0,1. Nf3 1... Nf6 2. e3 2... g6 3. b3 3... ...
4,2fe66e2e-1fcd-11ec-9f2a-09ff3c010001,True,blitz,checkmated,ivanmanshm,1d51-0,ivanmanshm,1. e4 1... c5 2. Nf3 2... d6 3. d4 3... c...
...,...,...,...,...,...,...,...,...
43105,d1d3361a-1b01-11ec-8206-09ff3c010001,True,blitz,checkmated,hleol,maximych26,hleol,1. e4 1... e6 2. d4 2... d5 3. Nd2 3... B...
43106,136ad418-1b04-11ec-8206-09ff3c010001,True,blitz,agreed,draw,maximych26,newauto,1. e4 1/2-1/2
43107,a3f3c176-1b05-11ec-8206-09ff3c010001,True,blitz,checkmated,gabduschew,gabduschew,maximych26,1. b3 1... d5 2. Bb2 2... Nf6 3. e3 3... ...
43108,03538990-1b07-11ec-8206-09ff3c010001,True,blitz,timeout,durin,maximych26,durin,0-1


In [54]:
# salva em csv
games_db.to_csv('games_db_46_54.csv',index = False)

In [ ]:
# Pesquisa de nomes no dataframe
#w = 'chesspartner123'
#texts = [*filter(lambda x: w in x, games_db['winner'])]
#print(*texts, sep=f'\n\n{"=" * 200}\n\n')